In [ ]:
from torch.nn.modules.container import ModuleList
from numpy.random import RandomState
import torch
import torchvision
import numpy as np
from torchvision import datasets, transforms

import torch.nn as nn 

import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CIFAR10

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from torch.utils.data import Subset
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC


In [ ]:
normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))

transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([transforms.ToTensor(), normalize]) 

cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
    
# We need two copies of this due to weird dataset api 
cifar_data_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)


Files already downloaded and verified
Files already downloaded and verified


### Applied GridSearchCV to find best hyperparameters, most of the default parameters turned out to be the best. 

# Boosting

## AdaBoosting

In [ ]:
def AdaBoosting(model):
  '''
  Initializes a base estimator and defines an AdaBoostClassifier predictor.
  '''

  if model == 'DecisionTreeClassifier':
    base_classifier = DecisionTreeClassifier(max_depth=8)
    n_estimators = 4  # after trial and error
  elif model == 'RandomForestClassifier':
    base_classifier = RandomForestClassifier(n_estimators=6)
    n_estimators = 8  # after trial and error
  elif model == 'LogisticRegression':
    base_classifier = LogisticRegression(solver='lbfgs')
    n_estimators = 6  # after trial and error
  elif model == 'SGDClassifier':
    base_classifier = SGDClassifier()
    n_estimators = 4  # after trial and error
  else:
    base_classifier = SVC(kernel='linear')
    n_estimators = 8  # after trial and error
  if model == 'SVC' or 'SGDClassifier':
    clf = AdaBoostClassifier(estimator=base_classifier, n_estimators=n_estimators, algorithm="SAMME")
  else:
    clf = AdaBoostClassifier(estimator=base_classifier, n_estimators=n_estimators)

  return clf

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

for model in ['SGDClassifier', 'RandomForestClassifier', 'SVC', 'DecisionTreeClassifier', 'LogisticRegression']:
  accs = []
  precision = []
  recall = []
  roc = []
  print("\n")
  print(f"For model: {model}")
  for seed in range(25):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes =  prng.permutation(np.arange(0,10))
    indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_data, indx_train)
    val_data = Subset(cifar_data_val, indx_val)

    print(f"Iteration: ", seed)
    print("\n")
    
    train_loader = torch.utils.data.DataLoader(train_data,
                                              batch_size=128, 
                                              shuffle=True)

    val_loader = torch.utils.data.DataLoader(val_data,
                                            batch_size=128, 
                                            shuffle=False)

    clf = AdaBoosting(model)
    for epoch in range(100):
      for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(images.shape[0], -1)
        labels = labels.numpy()
        clf.fit(images, labels)

    predictions = []
    targets = []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.reshape(images.shape[0], -1)
            unique = np.unique(labels)
            labels = labels.numpy()
            targets.extend(labels)
            pred = clf.predict(images)
            predictions.extend(pred)

    accuracy = accuracy_score(targets, predictions)
    precision_value = round(precision_score(targets, predictions, pos_label=unique[0]), 2)
    recall_value = round(recall_score(targets, predictions, pos_label=unique[0]), 2)
    confusion_mat = confusion_matrix(targets, predictions)
    auc_value = round(roc_auc_score(targets, predictions), 2)
    accs.append(accuracy)
    precision.append(precision_value)
    recall.append(recall_value)
    roc.append(auc_value)
            
    print("Accuracy:", accuracy)
    print("Confusion matrix: ")
    print(confusion_mat)
    print("Area under the curve: ", auc_value)
    print("Precision: ", precision_value)
    print("Recall: ", recall_value)
    print("\n")

  accs = np.array(accs)
  roc = np.array(roc)
  precision = np.array(precision)
  recall = np.array(recall)

  instances = 25

  print(f"Model: {model}")
  print(f'Accuracy over {instances} instances: %.2f +- %.2f'%(accs.mean(),accs.std()))
  print(f'AUC_ROC score over {instances} instances: %.2f +- %.2f'%(roc.mean(),roc.std()))
  print(f'Precision over {instances} instances: %.2f +- %.2f'%(precision.mean(),precision.std()))
  print(f'Recall over {instances} instances: %.2f +- %.2f'%(recall.mean(),recall.std()))





For model: SGDClassifier
Iteration:  0


Accuracy: 0.7175
Confusion matrix: 
[[126  74]
 [ 39 161]]
Area under the curve:  0.72
Precision:  0.69
Recall:  0.8


Iteration:  1


Accuracy: 0.63
Confusion matrix: 
[[ 96 104]
 [ 44 156]]
Area under the curve:  0.63
Precision:  0.69
Recall:  0.48


Iteration:  2


Accuracy: 0.635
Confusion matrix: 
[[123  77]
 [ 69 131]]
Area under the curve:  0.64
Precision:  0.64
Recall:  0.62


Iteration:  3


Accuracy: 0.61
Confusion matrix: 
[[128  72]
 [ 84 116]]
Area under the curve:  0.61
Precision:  0.62
Recall:  0.58


Iteration:  4


Accuracy: 0.6775
Confusion matrix: 
[[118  82]
 [ 47 153]]
Area under the curve:  0.68
Precision:  0.72
Recall:  0.59


Iteration:  5


Accuracy: 0.5875
Confusion matrix: 
[[106  94]
 [ 71 129]]
Area under the curve:  0.59
Precision:  0.6
Recall:  0.53


Iteration:  6


Accuracy: 0.6975
Confusion matrix: 
[[143  57]
 [ 64 136]]
Area under the curve:  0.7
Precision:  0.7
Recall:  0.68


Iteration:  7


Accuracy: 0.74

### LogisticRegression gives the best results for AdaBoosting


SVC: kernel type - 
- 'RBF', Acc for SVC over 5 instances: 0.66 +- 0.02
- 'poly', Acc for SVC over 5 instances: 0.57 +- 0.07
- 'linear', Acc for SVC over 5 instances: 0.66 +- 0.05
- 'sigmoid', Acc for SVC over 5 instances: 0.65 +- 0.03

## GradientBoosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

def GradBoosting(model):
  '''
  Initializes a base estimator and defines an GradientBoostClassifier predictor.
  '''

  if model == 'DecisionTreeClassifier':
    base_classifier = DecisionTreeClassifier(max_depth=8)
    n_estimators = 4
  elif model == 'RandomForestClassifier':
    base_classifier = RandomForestClassifier(n_estimators=6)
    n_estimators = 8
  elif model == 'LogisticRegression':
    base_classifier = LogisticRegression(solver='lbfgs')
    n_estimators = 6  # after trial and error
  elif model == 'SGDClassifier':
    base_classifier = SGDClassifier()
    n_estimators = 4  # after trial and error
  else:
    base_classifier = SVC(kernel='linear')
    n_estimators = 8
  clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)


  return clf


In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

for model in ['LogisticRegression', 'SGDClassifier', 'RandomForestClassifier', 'SVC', 'DecisionTreeClassifier']:
  accs = []
  precision = []
  recall = []
  roc = []

  print("\n")
  print(f"For mode: {model}")
  for seed in range(25):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes =  prng.permutation(np.arange(0,10))
    indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_data, indx_train)
    val_data = Subset(cifar_data_val, indx_val)

    print(f"Iteration: ", seed)
    print("\n")
    
    train_loader = torch.utils.data.DataLoader(train_data,
                                              batch_size=128, 
                                              shuffle=True)

    val_loader = torch.utils.data.DataLoader(val_data,
                                            batch_size=128, 
                                            shuffle=False)
  
    clf = GradBoosting(model)
    for i, (images, labels) in enumerate(train_loader):
      images = images.reshape(images.shape[0], -1)
      labels = labels.numpy()
      clf.fit(images, labels)

    predictions = []
    targets = []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.reshape(images.shape[0], -1)
            unique = np.unique(labels)
            labels = labels.numpy()
            targets.extend(labels)
            pred = clf.predict(images)
            predictions.extend(pred)

    accuracy = accuracy_score(targets, predictions)
    precision_value = round(precision_score(targets, predictions, pos_label=unique[0]), 2)
    recall_value = round(recall_score(targets, predictions, pos_label=unique[0]), 2)
    confusion_mat = confusion_matrix(targets, predictions)
    auc_value = round(roc_auc_score(targets, predictions), 2)
    accs.append(accuracy)
    precision.append(precision_value)
    recall.append(recall_value)
    roc.append(auc_value)
            
    print(f"For model: {model}")
    print("Accuracy:", accuracy)
    print("Confusion matrix: ")
    print(confusion_mat)
    print("Area under the curve: ", auc_value)
    print("Precision: ", precision_value)
    print("Recall: ", recall_value)
    print("\n")

  accs = np.array(accs)
  roc = np.array(roc)
  precision = np.array(precision)
  recall = np.array(recall)

  instances = 25

  print(f"Model: {model}")
  print(f'Accuracy over {instances} instances: %.2f +- %.2f'%(accs.mean(),accs.std()))
  print(f'AUC_ROC score over {instances} instances: %.2f +- %.2f'%(roc.mean(),roc.std()))
  print(f'Precision over {instances} instances: %.2f +- %.2f'%(precision.mean(),precision.std()))
  print(f'Recall over {instances} instances: %.2f +- %.2f'%(recall.mean(),recall.std()))






For mode: LogisticRegression
Iteration:  0


For model: LogisticRegression
Accuracy: 0.7375
Confusion matrix: 
[[148  52]
 [ 53 147]]
Area under the curve:  0.74
Precision:  0.74
Recall:  0.74


Iteration:  1


For model: LogisticRegression
Accuracy: 0.625
Confusion matrix: 
[[115  85]
 [ 65 135]]
Area under the curve:  0.62
Precision:  0.64
Recall:  0.57


Iteration:  2


For model: LogisticRegression
Accuracy: 0.6925
Confusion matrix: 
[[133  67]
 [ 56 144]]
Area under the curve:  0.69
Precision:  0.7
Recall:  0.66


Iteration:  3


For model: LogisticRegression
Accuracy: 0.6575
Confusion matrix: 
[[104  96]
 [ 41 159]]
Area under the curve:  0.66
Precision:  0.62
Recall:  0.8


Iteration:  4


For model: LogisticRegression
Accuracy: 0.6475
Confusion matrix: 
[[116  84]
 [ 57 143]]
Area under the curve:  0.65
Precision:  0.67
Recall:  0.58


Iteration:  5


For model: LogisticRegression
Accuracy: 0.5875
Confusion matrix: 
[[ 95 105]
 [ 60 140]]
Area under the curve:  0.59
Precision

### SGDClassifier and SVC give the best results for GradBoosting

## XGBoosting

In [ ]:
import xgboost as xgb

def XGBoosting(model):
  '''
  Initializes a base estimator and defines an GradientBoostClassifier predictor.
  '''

  if model == 'DecisionTreeClassifier':
    base_classifier = DecisionTreeClassifier(max_depth=8)
    n_estimators = 4
  elif model == 'RandomForestClassifier':
    base_classifier = RandomForestClassifier(n_estimators=6)
    n_estimators = 8
  elif model == 'LogisticRegression':
    base_classifier = LogisticRegression(solver='lbfgs')
    n_estimators = 6  # after trial and error
  elif model == 'SGDClassifier':
    base_classifier = SGDClassifier()
    n_estimators = 4  # after trial and error
  else:
    base_classifier = SVC(kernel='linear')
    n_estimators = 8
  clf = xgb.XGBClassifier(learning_rate=0.001, max_depth=1, n_estimators=100)


  return clf


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

for model in ['LogisticRegression', 'SGDClassifier', 'RandomForestClassifier', 'SVC', 'DecisionTreeClassifier']:
  accs = []
  precision = []
  recall = []
  roc = []
  print(f"For mode: {model}")
  for seed in range(25):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes =  prng.permutation(np.arange(0,10))
    indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_data, indx_train)
    val_data = Subset(cifar_data_val, indx_val)

    print(f"Iteration: ", seed)
    print("\n")
    
    train_loader = torch.utils.data.DataLoader(train_data,
                                              batch_size=128, 
                                              shuffle=True)

    val_loader = torch.utils.data.DataLoader(val_data,
                                            batch_size=128, 
                                            shuffle=False)
  
    clf = XGBoosting(model)
    for i, (images, labels) in enumerate(train_loader):
      images = images.reshape(images.shape[0], -1)
      labels = labels.numpy()
      labels = le.fit_transform(labels)
      clf.fit(images, labels)

    predictions = []
    targets = []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.reshape(images.shape[0], -1)
            unique = np.unique(labels)
            labels = labels.numpy()
            targets.extend(labels)
            pred = clf.predict(images)
            predictions.extend(pred)

    accuracy = accuracy_score(targets, predictions)
    print(labels)
    precision_value = round(precision_score(targets, predictions, pos_label=unique[0], average="macro"), 2)
    recall_value = round(recall_score(targets, predictions, pos_label=unique[0], average="macro"), 2)
    confusion_mat = confusion_matrix(targets, predictions)
    auc_value = round(roc_auc_score(targets, predictions), 2)
    accs.append(accuracy)
    precision.append(precision_value)
    recall.append(recall_value)
    roc.append(auc_value)
            
    print(f"For model: {model}")
    print("Accuracy:", accuracy)
    print("Confusion matrix: ")
    print(confusion_mat)
    print("Area under the curve: ", auc_value)
    print("Precision: ", precision_value)
    print("Recall: ", recall_value)
    print("\n")

  accs = np.array(accs)
  roc = np.array(roc)
  precision = np.array(precision)
  recall = np.array(recall)
  
  instances = 25

  print(f"Model: {model}")
  print(f'Accuracy over {instances} instances: %.2f +- %.2f'%(accs.mean(),accs.std()))
  print(f'AUC_ROC score over {instances} instances: %.2f +- %.2f'%(roc.mean(),roc.std()))
  print(f'Precision over {instances} instances: %.2f +- %.2f'%(precision.mean(),precision.std()))
  print(f'Recall over {instances} instances: %.2f +- %.2f'%(recall.mean(),recall.std()))



For mode: LogisticRegression
Iteration:  0


[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]
For model: LogisticRegression
Accuracy: 0.0
Confusion matrix: 
[[  0   0   0   0]
 [  0   0   0   0]
 [151  49   0   0]
 [ 84 116   0   0]]
Area under the curve:  0.67
Precision:  0.0
Recall:  0.0


Iteration:  1




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 9) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 9) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. U

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
For model: LogisticRegression
Accuracy: 0.2025
Confusion matrix: 
[[  0   0   0]
 [119  81   0]
 [120  80   0]]
Area under the curve:  0.5
Precision:  0.17
Recall:  0.14


Iteration:  2




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
For model: LogisticRegression
Accuracy: 0.0
Confusion matrix: 
[[  0   0   0   0]
 [  0   0   0   0]
 [141  59   0   0]
 [ 68 132   0   0]]
Area under the curve:  0.68
Precision:  0.0
Recall:  0.0


Iteration:  3




...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 6) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 6) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. U

[6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]
For model: DecisionTreeClassifier
Accuracy: 0.0
Confusion matrix: 
[[  0   0   0   0]
 [  0   0   0   0]
 [143  57   0   0]
 [133  67   0   0]]
Area under the curve:  0.52
Precision:  0.0
Recall:  0.0


Iteration:  24




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 6) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 6) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. U

[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
For model: DecisionTreeClassifier
Accuracy: 0.0
Confusion matrix: 
[[  0   0   0   0]
 [  0   0   0   0]
 [130  70   0   0]
 [ 95 105   0   0]]
Area under the curve:  0.59
Precision:  0.0
Recall:  0.0


Model: DecisionTreeClassifier
Accuracy over 25 instances: 0.10 +- 0.13
AUC_ROC score over 25 instances: 0.61 +- 0.06
Precision over 25 instances: 0.07 +- 0.09
Recall over 25 instances: 0.07 +- 0.09


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 7) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 7) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. U

### Used NuSVC which is similar to SVC that has a different parameter control. They are non-linear for image classification

In [ ]:
# GRAPH
# from sklearn.metrics import PrecisionRecallDisplay

# display = PrecisionRecallDisplay.from_estimator(
#     classifier, X_test, y_test, name="LinearSVC"
# )
# _ = display.ax_.set_title("2-class Precision-Recall curve")

# Stacking

## Final estimator - RandomForestClassifier

In [ ]:
estimators = [('lr', LogisticRegression()), ('knn', KNeighborsClassifier()), ('dt', DecisionTreeClassifier(), ('svc', SVC()))]
meta_estimator = RandomForestClassifier()  # 58

stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=meta_estimator)

accs = []
roc_score = []
precision = []
recall = []


for seed in range(25):
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 1000))
  classes =  prng.permutation(np.arange(0,10))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])


  train_data = Subset(cifar_data, indx_train)
  val_data = Subset(cifar_data_val, indx_val)

  print(f"Iteration: {seed}")
  print("\n")

  
  train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128, 
                                             shuffle=True)

  val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128, 
                                           shuffle=False)
  
  
  
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(images.shape[0], -1)
    labels = labels.numpy()
    unique = np.unique(labels)
    stacking_classifier.fit(images, labels)

  predictions = []
  targets = []
  with torch.no_grad():
      for images, labels in val_loader:
          images = images.reshape(images.shape[0], -1)
          labels = labels.numpy()
          targets.extend(labels)
          pred = stacking_classifier.predict(images)
          predictions.extend(pred)
  print(len(predictions))
  accuracy = accuracy_score(targets, predictions)
  confusion_mat = confusion_matrix(targets, predictions)
  precision_value = round(precision_score(targets, predictions, pos_label=unique[0]), 2)
  recall_value = round(recall_score(targets, predictions, pos_label=unique[0]), 2)
  auc = round(roc_auc_score(targets, predictions), 2)
  print("Accuracy:", accuracy)
  print("Confusion matrix: ")
  print(confusion_mat)
  print("Area under the curve: ", auc)
  print("Precision: ", precision_value)
  print("Recall: ", recall_value)
  print("\n")
  accs.append(accuracy)
  roc_score.append(auc)
  precision.append(precision_value)
  recall.append(recall_value)

accs = np.array(accs)
roc_score = np.array(roc_score)
precision = np.array(precision)
recall = np.array(recall)

instances = 25

print(f"Model: {model}")
print(f'Accuracy over {instances} instances: %.2f +- %.2f'%(accs.mean(),accs.std()))
print(f'AUC_ROC score over {instances} instances: %.2f +- %.2f'%(roc.mean(),roc.std()))
print(f'Precision over {instances} instances: %.2f +- %.2f'%(precision.mean(),precision.std()))
print(f'Recall over {instances} instances: %.2f +- %.2f'%(recall.mean(),recall.std()))


Iteration: 0


400
Accuracy: 0.65
Confusion matrix: 
[[119  81]
 [ 59 141]]
Area under the curve:  0.65
Precision:  0.67
Recall:  0.6


Iteration: 1


400
Accuracy: 0.545
Confusion matrix: 
[[111  89]
 [ 93 107]]
Area under the curve:  0.54
Precision:  0.54
Recall:  0.56


Iteration: 2


400
Accuracy: 0.5975
Confusion matrix: 
[[123  77]
 [ 84 116]]
Area under the curve:  0.6
Precision:  0.59
Recall:  0.62


Iteration: 3


400
Accuracy: 0.6175
Confusion matrix: 
[[126  74]
 [ 79 121]]
Area under the curve:  0.62
Precision:  0.61
Recall:  0.63


Iteration: 4


400
Accuracy: 0.6175
Confusion matrix: 
[[108  92]
 [ 61 139]]
Area under the curve:  0.62
Precision:  0.64
Recall:  0.54


Iteration: 5


400
Accuracy: 0.53
Confusion matrix: 
[[102  98]
 [ 90 110]]
Area under the curve:  0.53
Precision:  0.53
Recall:  0.51


Iteration: 6


400
Accuracy: 0.5475
Confusion matrix: 
[[106  94]
 [ 87 113]]
Area under the curve:  0.55
Precision:  0.55
Recall:  0.53


Iteration: 7


400
Accuracy: 0.727

## Final estimator - SVC

In [ ]:
estimators = [('lr', LogisticRegression()), ('knn', KNeighborsClassifier()), ('dt', DecisionTreeClassifier(), ('rfc', RandomForestClassifier()))]
meta_estimator = SVC()  # 66

stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=meta_estimator)

accs = []
roc_score = []
precision = []
recall = []


for seed in range(25):
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 1000))
  classes =  prng.permutation(np.arange(0,10))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])


  train_data = Subset(cifar_data, indx_train)
  val_data = Subset(cifar_data_val, indx_val)

  print(f"Iteration: {seed}")

  train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128, 
                                             shuffle=True)

  val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128, 
                                           shuffle=False)
  
  
  
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(images.shape[0], -1)
    labels = labels.numpy()
    unique = np.unique(labels)
    stacking_classifier.fit(images, labels)

  predictions = []
  targets = []
  with torch.no_grad():
      for images, labels in val_loader:
          images = images.reshape(images.shape[0], -1)
          labels = labels.numpy()
          targets.extend(labels)
          pred = stacking_classifier.predict(images)
          predictions.extend(pred)
  print(len(predictions))
  accuracy = accuracy_score(targets, predictions)
  confusion_mat = confusion_matrix(targets, predictions)
  precision_value = round(precision_score(targets, predictions, pos_label=unique[0]), 2)
  recall_value = round(recall_score(targets, predictions, pos_label=unique[0]), 2)
  auc = round(roc_auc_score(targets, predictions), 2)
  print("Accuracy:", accuracy)
  print("Confusion matrix: ")
  print(confusion_mat)
  print("Area under the curve: ", auc)
  print("Precision: ", precision_value)
  print("Recall: ", recall_value)
  print("\n")
  accs.append(accuracy)
  roc_score.append(auc)
  precision.append(precision_value)
  recall.append(recall_value)

accs = np.array(accs)
roc_score = np.array(roc_score)
precision = np.array(precision)
recall = np.array(recall)

instances = 25

print(f"Model: {model}")
print(f'Accuracy over {instances} instances: %.2f +- %.2f'%(accs.mean(),accs.std()))
print(f'AUC_ROC score over {instances} instances: %.2f +- %.2f'%(roc.mean(),roc.std()))
print(f'Precision over {instances} instances: %.2f +- %.2f'%(precision.mean(),precision.std()))
print(f'Recall over {instances} instances: %.2f +- %.2f'%(recall.mean(),recall.std()))




Iteration: 0
400
Accuracy: 0.78
Confusion matrix: 
[[166  34]
 [ 54 146]]
Area under the curve:  0.78
Precision:  0.75
Recall:  0.83


Iteration: 1
400
Accuracy: 0.51
Confusion matrix: 
[[163  37]
 [159  41]]
Area under the curve:  0.51
Precision:  0.51
Recall:  0.82


Iteration: 2
400
Accuracy: 0.59
Confusion matrix: 
[[ 97 103]
 [ 61 139]]
Area under the curve:  0.59
Precision:  0.61
Recall:  0.48


Iteration: 3
400
Accuracy: 0.7325
Confusion matrix: 
[[120  80]
 [ 27 173]]
Area under the curve:  0.73
Precision:  0.82
Recall:  0.6


Iteration: 4
400
Accuracy: 0.69
Confusion matrix: 
[[120  80]
 [ 44 156]]
Area under the curve:  0.69
Precision:  0.73
Recall:  0.6


Iteration: 5
400
Accuracy: 0.595
Confusion matrix: 
[[ 86 114]
 [ 48 152]]
Area under the curve:  0.6
Precision:  0.64
Recall:  0.43


Iteration: 6
400
Accuracy: 0.7375
Confusion matrix: 
[[132  68]
 [ 37 163]]
Area under the curve:  0.74
Precision:  0.78
Recall:  0.66


Iteration: 7
400
Accuracy: 0.79
Confusion matrix: 
[[

## Final estimator - LogisticRegression

In [ ]:
estimators = [('rfc', RandomForestClassifier()), ('knn', KNeighborsClassifier()), ('dt', DecisionTreeClassifier(), ('svc', SVC()))]
meta_estimator = LogisticRegression()  # 61


stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=meta_estimator)

accs = []
roc_score = []
precision = []
recall = []


for seed in range(25):
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 1000))
  classes =  prng.permutation(np.arange(0,10))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])


  train_data = Subset(cifar_data, indx_train)
  val_data = Subset(cifar_data_val, indx_val)

  print(f"Iteration: {seed}")
  print("\n")

  train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128, 
                                             shuffle=True)

  val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128, 
                                           shuffle=False)
  
  
  
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(images.shape[0], -1)
    labels = labels.numpy()
    unique = np.unique(labels)
    stacking_classifier.fit(images, labels)

  predictions = []
  targets = []
  with torch.no_grad():
      for images, labels in val_loader:
          images = images.reshape(images.shape[0], -1)
          labels = labels.numpy()
          targets.extend(labels)
          pred = stacking_classifier.predict(images)
          predictions.extend(pred)
  print(len(predictions))
  accuracy = accuracy_score(targets, predictions)
  confusion_mat = confusion_matrix(targets, predictions)
  precision_value = round(precision_score(targets, predictions, pos_label=unique[0]), 2)
  recall_value = round(recall_score(targets, predictions, pos_label=unique[0]), 2)
  auc = round(roc_auc_score(targets, predictions), 2)
  print("Accuracy:", accuracy)
  print("Confusion matrix: ")
  print(confusion_mat)
  print("Area under the curve: ", auc)
  print("Precision: ", precision_value)
  print("Recall: ", recall_value)
  print("\n")
  accs.append(accuracy)
  roc_score.append(auc)
  precision.append(precision_value)
  recall.append(recall_value)

accs = np.array(accs)
roc_score = np.array(roc_score)
precision = np.array(precision)
recall = np.array(recall)
  
instances = 25

print(f"Model: {model}")
print(f'Accuracy over {instances} instances: %.2f +- %.2f'%(accs.mean(),accs.std()))
print(f'AUC_ROC score over {instances} instances: %.2f +- %.2f'%(roc.mean(),roc.std()))
print(f'Precision over {instances} instances: %.2f +- %.2f'%(precision.mean(),precision.std()))
print(f'Recall over {instances} instances: %.2f +- %.2f'%(recall.mean(),recall.std()))



Iteration: 0


400
Accuracy: 0.76
Confusion matrix: 
[[159  41]
 [ 55 145]]
Area under the curve:  0.76
Precision:  0.74
Recall:  0.8


Iteration: 1


400
Accuracy: 0.5325
Confusion matrix: 
[[112  88]
 [ 99 101]]
Area under the curve:  0.53
Precision:  0.53
Recall:  0.56


Iteration: 2


400
Accuracy: 0.5975
Confusion matrix: 
[[102  98]
 [ 63 137]]
Area under the curve:  0.6
Precision:  0.62
Recall:  0.51


Iteration: 3


400
Accuracy: 0.695
Confusion matrix: 
[[119  81]
 [ 41 159]]
Area under the curve:  0.7
Precision:  0.74
Recall:  0.6


Iteration: 4


400
Accuracy: 0.6475
Confusion matrix: 
[[108  92]
 [ 49 151]]
Area under the curve:  0.65
Precision:  0.69
Recall:  0.54


Iteration: 5


400
Accuracy: 0.6125
Confusion matrix: 
[[107  93]
 [ 62 138]]
Area under the curve:  0.61
Precision:  0.63
Recall:  0.54


Iteration: 6


400
Accuracy: 0.635
Confusion matrix: 
[[126  74]
 [ 72 128]]
Area under the curve:  0.64
Precision:  0.64
Recall:  0.63


Iteration: 7


400
Accuracy: 0.71
C

## Final estimator - KNeighborsClassifier

In [ ]:
estimators = [('lr', LogisticRegression()), ('rfc', RandomForestClassifier()), ('dt', DecisionTreeClassifier(), ('svc', SVC()))]
meta_estimator = KNeighborsClassifier()  # 64

stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=meta_estimator)

accs = []
roc_score = []
precision = []
recall = []


for seed in range(25):
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 1000))
  classes =  prng.permutation(np.arange(0,10))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])


  train_data = Subset(cifar_data, indx_train)
  val_data = Subset(cifar_data_val, indx_val)

  print(f"Iteration: {seed}")  
  print("\n")
  train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128, 
                                             shuffle=True)

  val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128, 
                                           shuffle=False)
  
  
  
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(images.shape[0], -1)
    labels = labels.numpy()
    unique = np.unique(labels)
    stacking_classifier.fit(images, labels)

  predictions = []
  targets = []
  with torch.no_grad():
      for images, labels in val_loader:
          images = images.reshape(images.shape[0], -1)
          labels = labels.numpy()
          targets.extend(labels)
          pred = stacking_classifier.predict(images)
          predictions.extend(pred)
  print(len(predictions))
  accuracy = accuracy_score(targets, predictions)
  confusion_mat = confusion_matrix(targets, predictions)
  precision_value = round(precision_score(targets, predictions, pos_label=unique[0]), 2)
  recall_value = round(recall_score(targets, predictions, pos_label=unique[0]), 2)
  auc = round(roc_auc_score(targets, predictions), 2)
  print("Accuracy:", accuracy)
  print("Confusion matrix: ")
  print(confusion_mat)
  print("Area under the curve: ", auc)
  print("Precision: ", precision_value)
  print("Recall: ", recall_value)
  print("\n")
  accs.append(accuracy)
  roc_score.append(auc)
  precision.append(precision_value)
  recall.append(recall_value)

accs = np.array(accs)
roc_score = np.array(roc_score)
precision = np.array(precision)
recall = np.array(recall)

instances = 25

print(f"Model: {model}")
print(f'Accuracy over {instances} instances: %.2f +- %.2f'%(accs.mean(),accs.std()))
print(f'AUC_ROC score over {instances} instances: %.2f +- %.2f'%(roc.mean(),roc.std()))
print(f'Precision over {instances} instances: %.2f +- %.2f'%(precision.mean(),precision.std()))
print(f'Recall over {instances} instances: %.2f +- %.2f'%(recall.mean(),recall.std()))


Iteration: 0


400
Accuracy: 0.78
Confusion matrix: 
[[157  43]
 [ 45 155]]
Area under the curve:  0.78
Precision:  0.78
Recall:  0.78


Iteration: 1


400
Accuracy: 0.5225
Confusion matrix: 
[[ 78 122]
 [ 69 131]]
Area under the curve:  0.52
Precision:  0.53
Recall:  0.39


Iteration: 2


400
Accuracy: 0.59
Confusion matrix: 
[[122  78]
 [ 86 114]]
Area under the curve:  0.59
Precision:  0.59
Recall:  0.61


Iteration: 3


400
Accuracy: 0.675
Confusion matrix: 
[[125  75]
 [ 55 145]]
Area under the curve:  0.68
Precision:  0.69
Recall:  0.62


Iteration: 4


400
Accuracy: 0.63
Confusion matrix: 
[[136  64]
 [ 84 116]]
Area under the curve:  0.63
Precision:  0.62
Recall:  0.68


Iteration: 5


400
Accuracy: 0.585
Confusion matrix: 
[[116  84]
 [ 82 118]]
Area under the curve:  0.58
Precision:  0.59
Recall:  0.58


Iteration: 6


400
Accuracy: 0.7325
Confusion matrix: 
[[126  74]
 [ 33 167]]
Area under the curve:  0.73
Precision:  0.79
Recall:  0.63


Iteration: 7


400
Accuracy: 0.7825

## Final estimator - DecisionTreeClassifier

In [ ]:
estimators = [('lr', LogisticRegression()), ('knn', KNeighborsClassifier()), ('rfc', RandomForestClassifier(), ('svc', SVC()))]
meta_estimator = DecisionTreeClassifier()  # 57

stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=meta_estimator)

accs = []
roc_score = []
precision = []
recall = []


for seed in range(25):
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 1000))
  classes =  prng.permutation(np.arange(0,10))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])


  train_data = Subset(cifar_data, indx_train)
  val_data = Subset(cifar_data_val, indx_val)

  print(f"Iteration: {seed}")  
  print("\n")
  train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128, 
                                             shuffle=True)

  val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128, 
                                           shuffle=False)
  
  
  
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(images.shape[0], -1)
    labels = labels.numpy()
    unique = np.unique(labels)
    stacking_classifier.fit(images, labels)

  predictions = []
  targets = []
  with torch.no_grad():
      for images, labels in val_loader:
          images = images.reshape(images.shape[0], -1)
          labels = labels.numpy()
          targets.extend(labels)
          pred = stacking_classifier.predict(images)
          predictions.extend(pred)
  print(len(predictions))
  accuracy = accuracy_score(targets, predictions)
  confusion_mat = confusion_matrix(targets, predictions)
  precision_value = round(precision_score(targets, predictions, pos_label=unique[0]), 2)
  recall_value = round(recall_score(targets, predictions, pos_label=unique[0]), 2)
  auc = round(roc_auc_score(targets, predictions), 2)
  print("Accuracy:", accuracy)
  print("Confusion matrix: ")
  print(confusion_mat)
  print("Area under the curve: ", auc)
  print("Precision: ", precision_value)
  print("Recall: ", recall_value)
  print("\n")
  accs.append(accuracy)
  roc_score.append(auc)
  precision.append(precision_value)
  recall.append(recall_value)

accs = np.array(accs)
roc_score = np.array(roc_score)
precision = np.array(precision)
recall = np.array(recall)

instances = 25

print(f"Model: {model}")
print(f'Accuracy over {instances} instances: %.2f +- %.2f'%(accs.mean(),accs.std()))
print(f'AUC_ROC score over {instances} instances: %.2f +- %.2f'%(roc.mean(),roc.std()))
print(f'Precision over {instances} instances: %.2f +- %.2f'%(precision.mean(),precision.std()))
print(f'Recall over {instances} instances: %.2f +- %.2f'%(recall.mean(),recall.std()))


Iteration: 0


400
Accuracy: 0.6175
Confusion matrix: 
[[103  97]
 [ 56 144]]
Area under the curve:  0.62
Precision:  0.65
Recall:  0.52


Iteration: 1


400
Accuracy: 0.475
Confusion matrix: 
[[ 87 113]
 [ 97 103]]
Area under the curve:  0.48
Precision:  0.47
Recall:  0.44


Iteration: 2


400
Accuracy: 0.565
Confusion matrix: 
[[112  88]
 [ 86 114]]
Area under the curve:  0.56
Precision:  0.57
Recall:  0.56


Iteration: 3


400
Accuracy: 0.5525
Confusion matrix: 
[[115  85]
 [ 94 106]]
Area under the curve:  0.55
Precision:  0.55
Recall:  0.57


Iteration: 4


400
Accuracy: 0.62
Confusion matrix: 
[[110  90]
 [ 62 138]]
Area under the curve:  0.62
Precision:  0.64
Recall:  0.55


Iteration: 5


400
Accuracy: 0.515
Confusion matrix: 
[[110  90]
 [104  96]]
Area under the curve:  0.52
Precision:  0.51
Recall:  0.55


Iteration: 6


400
Accuracy: 0.6025
Confusion matrix: 
[[120  80]
 [ 79 121]]
Area under the curve:  0.6
Precision:  0.6
Recall:  0.6


Iteration: 7


400
Accuracy: 0.71
C

#Results:
## final-estimator:
- RFC: 
  -  Accuracy over 5 instances: 0.58 +- 0.05
  -  AUC_ROC score over 5 instances: 0.58 +- 0.05
  -  Precision over 5 instances: 0.59 +- 0.07
  -  Recall over 5 instances: 0.54 +- 0.06

- SVC:
  - Accuracy over 5 instances: 0.64 +- 0.05
  - AUC_ROC score over 5 instances: 0.64 +- 0.05
  - Precision over 5 instances: 0.67 +- 0.11
  - Recall over 5 instances: 0.63 +- 0.10

- Logistic:
  - Accuracy over 5 instances: 0.60 +- 0.09
  - AUC_ROC score over 5 instances: 0.60 +- 0.09
  - Precision over 5 instances: 0.63 +- 0.12
  - Recall over 5 instances: 0.51 +- 0.07

- Kneighbors:
  - Accuracy over 5 instances: 0.65 +- 0.06
  - AUC_ROC score over 5 instances: 0.65 +- 0.06
  - Precision over 5 instances: 0.67 +- 0.08
  - Recall over 5 instances: 0.61 +- 0.10

- DecisionTree:
  - Accuracy over 5 instances: 0.57 +- 0.05
  - AUC_ROC score over 5 instances: 0.57 +- 0.05
  - Precision over 5 instances: 0.57 +- 0.05
  - Recall over 5 instances: 0.58 +- 0.10